<a href="https://colab.research.google.com/github/palswayam5/LLM-Model---Detect-AI-Generated-Text/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --quiet "tensorflow-text==2.11.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 46.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.4.4 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but 

In [2]:
pip install imbalanced-learn

In [3]:
!pip install -U tensorflow
!pip install -U tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  

In [4]:
import numpy as np
import pandas as pd
import imblearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense,Input,Dropout
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns

EDA of Dataset

In [5]:
train_prompts = pd.read_csv('/content/drive/MyDrive/llm-detect-ai-generated-text/train_prompts.csv')
train_essay = pd.read_csv('/content/drive/MyDrive/llm-detect-ai-generated-text/train_essays.csv')

In [6]:
train_prompts.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [7]:
train_essay.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [8]:
train_essay.shape

(1378, 4)

In [9]:
train_essay['generated'].value_counts()

0    1375
1       3
Name: generated, dtype: int64

In [10]:
train_essay.isnull().value_counts()

id     prompt_id  text   generated
False  False      False  False        1378
dtype: int64

In [11]:
X = train_essay['text']
y = train_essay['generated']

In [12]:
labels = ['generated','not-generated']

In [13]:
print(X.shape,y.shape)

(1378,) (1378,)


Preprocessing of Data
1. Convert it into embeddings using BERT encoder
2. Oversample that that data using oversampler of scikit-learn

In [14]:
external_data = pd.read_csv('/content/drive/MyDrive/llm-detect-ai-generated-text/daigt_external_dataset.csv')

In [15]:
external_data.head()

,id,text,instructions,source_text
0,6060D28C05B6,Some schools in United States ofter classes fr...,\nTask: Write a persuasive essay on whether or...,\nWhen considering the pros and cons of attend...
1,60623DB5DE7A,"Four-day work week, a remarkable idea to conse...",\nTask: Research the advantages and disadvanta...,\nOne of the primary arguments for implementin...
2,607A39D981DE,Students and their families should consider an...,\nTask: \n\n1. Talk to your parents before tak...,\nBefore making any decisions about getting in...
3,60ACDFA1609E,Agree you will never grow if something beyond ...,\nTask: Write an essay discussing the benefits...,"\nRalph Waldo Emerson once said, ""Go confident..."
4,60AE13D3F07B,I think our character traits are formed by inf...,\nTask: Research and discuss how character tra...,\nHuman character traits are shaped by a wide ...


In [16]:
X_external = external_data['text']

In [17]:
y_external = np.zeros([X_external.shape[0]])

In [18]:
for i in range(X_external.shape[0]):
  y_external[i] = 1

In [19]:
X_external =  pd.DataFrame(X_external,columns = ['text'])
y_external = pd.DataFrame(y_external,columns = ['generated'])

In [20]:
X_external.head()

,text
0,Some schools in United States ofter classes fr...
1,"Four-day work week, a remarkable idea to conse..."
2,Students and their families should consider an...
3,Agree you will never grow if something beyond ...
4,I think our character traits are formed by inf...


In [21]:
X = pd.DataFrame(X,columns = ['text'])
y = pd.DataFrame(y,columns = ['generated'])

In [22]:
X_new = pd.concat([X,X_external],axis = 0)
y_new = pd.concat([y,y_external],axis = 0)

In [23]:
print(X.shape,y.shape,X_external.shape,y_external.shape)

(1378, 1) (1378, 1) (2421, 1) (2421, 1)


In [24]:
print(X_new.shape,y_new.shape)

(3799, 1) (3799, 1)


Balanced Dataset

In [25]:
y_new['generated'].value_counts()

1.0    2424
0.0    1375
Name: generated, dtype: int64

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X_new,y_new)

In [27]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(2849, 1) (950, 1) (2849, 1) (950, 1)


In [28]:
#oversample = RandomOverSampler(sampling_strategy = 0.5)

In [29]:
bert_preprocess = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Function to implement embeddings

In [30]:
#def get_sentence_embeding(sentences):
#    preprocessed_text = bert_preprocess(sentences)
#    return bert_encoder(preprocessed_text)['pooled_output']

Bert Model Implementation

In [31]:
#bert layers
text_input = Input(shape =(),dtype = tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

#neural network layers
x = Dropout(rate = 0.1)(outputs['pooled_output'])
y = Dense(1,activation = 'sigmoid')(x)

model = Model(inputs = [text_input],outputs = [y])

In [32]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['input_1[0][0]']             
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [33]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name = "accuracy"),
    tf.keras.metrics.Precision(name = "precision"),
    tf.keras.metrics.Recall(name = "recall")
]

model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = METRICS)

In [34]:
model.fit(X_train,y_train,epochs = 5)

Epoch 1/5
90/90 [==============================] - 1394s 15s/step - loss: 0.5443 - accuracy: 0.7389 - precision: 0.7271 - recall: 0.9485
Epoch 2/5
90/90 [==============================] - 1372s 15s/step - loss: 0.4254 - accuracy: 0.8435 - precision: 0.8252 - recall: 0.9589
Epoch 3/5
90/90 [==============================] - 1372s 15s/step - loss: 0.3473 - accuracy: 0.8866 - precision: 0.8737 - recall: 0.9622
Epoch 4/5
90/90 [==============================] - 1361s 15s/step - loss: 0.3062 - accuracy: 0.9063 - precision: 0.8955 - recall: 0.9666
Epoch 5/5
90/90 [==============================] - 1385s 15s/step - loss: 0.2721 - accuracy: 0.9186 - precision: 0.9134 - recall: 0.9644


In [37]:
essay_test = pd.read_csv('/content/drive/MyDrive/llm-detect-ai-generated-text/test_essays.csv')

In [38]:
essay_test.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [39]:
id = essay_test['id']
X_test = essay_test['text']

In [41]:
y_test = model.predict(X_test)

1/1 [==============================] - 2s 2s/step


In [42]:
submission = pd.read_csv('/content/drive/MyDrive/llm-detect-ai-generated-text/sample_submission.csv')
submission.head()

,id,generated
0,0000aaaa,0.1
1,1111bbbb,0.9
2,2222cccc,0.4


In [43]:
y_test = pd.DataFrame(y_test,columns = ['generated'])
id = pd.DataFrame(id,columns = ['id'])

In [45]:
sub = pd.concat([id,y_test],axis = 1)

In [46]:
sub.to_csv('submission.csv')